#### **Gruppo 5.b**: Caponi Marco (matricola: 508773) - Ceneda Gianluca (matricola: 488257)

# ANALISI E REVISIONE DEL PROGETTO LARSPLITTING 2D 

## CLASSE REFACTORING: congruence

Variabili utili per testare il funzionamento


In [3]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads


In [6]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);    #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);   #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];             #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]    #intersezioni tra bounding box

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

## Versione iniziale di congruence

funzione che prende in ingresso un modello di Lar, restituendo una funzione di base denominata hcat che concatena due array lungo due dimensioni.

In [4]:
function congruence(model)
	W,EW = model
	# congruent vertices
	balltree = NearestNeighbors.BallTree(W)
	r = 0.0000000001
	near = Array{Any}(undef, size(W,2))
@sync	for k=1:size(W,2)
@views	near[k] = union([NearestNeighbors.inrange(balltree, W[:,k], r, true)]...)
	end
	near = map(sort,near)  # check !!!
	for k=1:size(W,2)
		W[:,k] = W[:,near[k][1]]
	end
	pointidx = [ near[k][1] for k=1:size(W,2) ]  # check !!
	invidx = OrderedDict(zip(1:length(pointidx), pointidx))
	V = [W[:,k] for k=1:length(pointidx)]
	# congruent edges
	EV = []
@async	for e in (EW)
@views		newedge = [invidx[e[1]],invidx[e[2]]]
		if newedge[1] !== newedge[2]
			push!(EV,newedge)
		end
	end
	EV = [EV[h] for h=1:length(EV) if length(EV[h])==2]
	EV = convert(Lar.Cells, EV)
	#W,EW = Lar.simplifyCells(V,EV)
	return hcat(V...),EV
end




congruence (generic function with 1 method)

In [14]:
@btime congruence((V, EV))       #36,683 μs

  36.683 μs (161 allocations: 8.92 KiB)


([0.0 1.0 … 0.0 2.0; 0.0 0.0 … 1.0 1.0], [[1, 2], [2, 3], [3, 4], [4, 1], [1, 5]])

### Controllo se la funzione è type unstable

In [ ]:
@code_warntype congruence((V,EV))

La funzione è type stable in quanto ritorna la stringa:

Body::Tuple{Array,Any}

## Versione modificata di congruence


In [9]:
using Base.Threads
function congruenceMOD(model)
    W,EW = model
    n = size(W,2)
    balltree = NearestNeighbors.BallTree(W)
    r = 0.0000000001
    near = Array{Any}(undef, n)
    @inbounds @threads for k=1:n
        near[k] = NearestNeighbors.inrange(balltree, W[:,k], r, true)
    end
    near = map(sort,near) 
    @inbounds @threads for k=1:n
        W[:,k] = W[:,near[k][1]]
    end
    pointidx = Array{Int64}(undef, n)
    @inbounds @threads for k=1:n
         pointidx[k] = near[k][1] 
    end
    l = length(pointidx)
    invidx = OrderedDict(zip(1:l, pointidx))
    V = Array{Array{Float64,1}}(undef, l)
    @inbounds @threads for k=1:l
        V[k] = W[:,k] 
    end
    
    EV = []
    m = length(EW)
    @inbounds for i = 1:m
        newedge = [invidx[EW[i][1]],invidx[EW[i][2]]]
        if newedge[1] !== newedge[2]
            push!(EV,newedge)
        end
    end
    EV = [EV[h] for h=1:length(EV) if length(EV[h])==2]
    EV = convert(Lar.Cells, EV)
    return hcat(V...),EV
end


congruenceMOD (generic function with 1 method)

In [13]:
@btime congruenceMOD((V, EV))       #19,516 μs

  19.516 μs (184 allocations: 9.94 KiB)


([0.0 1.0 … 0.0 2.0; 0.0 0.0 … 1.0 1.0], [[1, 2], [2, 3], [3, 4], [4, 1], [1, 5]])

### Benchmark della funzione iniziale e modificata

funzione iniziale:

In [15]:
@benchmark congruence((V, EV))   

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  36.842 μs …   9.054 ms  ┊ GC (min … max): 0.00% … 98.53%
 Time  (median):     38.223 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   45.952 μs ± 112.596 μs  ┊ GC (mean ± σ):  3.22% ±  1.40%

  █▅▂▁                                                         ▁
  █████▇▇▆▇▆▆▇▇▇███▇▇▇█▆▆▆▆▆▆▆▆▆▆▅▅▅▅▆▆▅▅▆▃▅▄▄▅▅▆▄▅▅▁▄▄▅▄▄▃▄▃▅ █
  36.8 μs       Histogram: log(frequency) by time       171 μs <

 Memory estimate: 8.92 KiB, allocs estimate: 161.

funzione modificata:

In [16]:
@benchmark congruenceMOD((V, EV)) 

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  19.597 μs …  60.177 ms  ┊ GC (min … max):  0.00% … 99.89%
 Time  (median):     21.919 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   30.737 μs ± 601.869 μs  ┊ GC (mean ± σ):  19.56% ±  1.00%

  ▇█▇▆▅▄▂▁                                                     ▂
  █████████▆▇▇▇▇▆▇▆▆▇▆▇▆▇▆▆▅▆▅▆▅▅▅▅▄▅▄▅▅▅▄▄▅▄▅▄▃▂▃▅▄▂▄▃▄▄▅▃▄▅▄ █
  19.6 μs       Histogram: log(frequency) by time      86.9 μs <

 Memory estimate: 9.94 KiB, allocs estimate: 184.

## TEST

In [ ]:
@testset "congruence" begin
    V1 = [0.0 1.0 ; 
         0.0 0.0]
    EV1 = [[1,2]]
    a, b = splitting.congruence((V1,EV1))
    @test a == V1
    @test b == EV1

    V1 = [0.0 1.0 0.99999999999999999 ; 
         0.0 0.0 0.0]
    EV1 = [[1,2], [1,3]]
    a, b = splitting.congruence((V1,EV1))
    @test b == [[1, 2], [1, 2]] 
    
end

![test di boxcovering](https://github.com/MarcoCap13/LAR-SPLITTING-2D-5.b-/blob/main/docs/test/boxcovering_test.png?raw=true)